In [1]:
import json
from pathlib import Path
import pandas as pd

from common import clean_df

DATA_DIR = Path("../data")
INPUT_DIR = DATA_DIR / "input"
EMBEDDING_EVAL_DIR = DATA_DIR / "embedding_eval"

df = pd.read_parquet(EMBEDDING_EVAL_DIR / "Angriff_Wolf.parquet")
df

,result,gecko@001,gecko@001_distance,gecko@001_order,gecko@002_tasktype,gecko@002_tasktype_distance,gecko@002_tasktype_order,gecko@002,gecko@002_distance,gecko@002_order,...,gecko@003,gecko@003_distance,gecko@003_order,geckomultilingual@001_tasktype,geckomultilingual@001_tasktype_distance,geckomultilingual@001_tasktype_order,geckomultilingual@001,geckomultilingual@001_distance,geckomultilingual@001_order,source
0,KRIEG IN DER UKRAINE - TAG 121 - AKTUELLE LAGE...,"[-0.02481548674404621, -0.002519130241125822, ...",0.78,15,"[-0.021496394649147987, 0.0003441041917540133,...",0.79,19,"[-0.009350113570690155, -0.010239853523671627,...",0.52,45,...,"[0.028743164613842964, -0.05943309888243675, -...",0.84,35,"[0.017298677936196327, -0.007602742873132229, ...",0.84,51,"[0.004083053208887577, -0.004955140873789787, ...",0.85,49,semantic
1,SCHWERE ZERSTÖRUNGEN NACH LUFTANGRIFF AUF GEBÄ...,"[-0.040524911135435104, -0.009754134342074394,...",0.76,5,"[-0.03512757271528244, -0.0009582368656992912,...",0.78,8,"[-0.021815333515405655, -0.007010355591773987,...",0.50,18,...,"[0.03647651523351669, -0.00800247397273779, -0...",0.86,40,"[-0.016848251223564148, 0.0022234604693949223,...",0.83,47,"[-0.037596337497234344, -0.0020669468212872744...",0.85,49,semantic
2,KRIEG IN DER UKRAINE - TAG 572 (17.09.) - AKTU...,"[-0.007534890901297331, 0.003501564497128129, ...",0.77,9,"[-0.01101269107311964, 0.0056565082632005215, ...",0.81,50,"[-0.0005561228026635945, -0.010257836431264877...",0.52,45,...,"[0.03759535402059555, -0.01924535259604454, -0...",0.88,49,"[0.0072788349352777, -0.011984610930085182, 0....",0.81,37,"[-0.004619269631803036, -0.010121668688952923,...",0.81,36,semantic
3,SITUATION NACH LUFTANGRIFF - 9. Mar 2023 in Ki...,"[-0.015058608725667, 0.016938960179686546, -0....",0.76,5,"[-0.015390072949230671, 0.020570751279592514, ...",0.81,50,"[-0.00311960163526237, 0.00595601461827755, -0...",0.50,18,...,"[0.07284429669380188, 0.004694993142038584, -0...",0.91,63,"[0.014095309190452099, -0.01135972235351801, 0...",0.86,62,"[-0.0036410547327250242, -0.006985177285969257...",0.89,69,semantic
4,KRIEG IN DER UKRAINE - TAG 538 (15.08.) - AKTU...,"[-0.011629147455096245, 0.009755927138030529, ...",0.78,15,"[-0.01437787152826786, 0.010400270111858845, 0...",0.80,33,"[-0.0009389095939695835, -0.002985538449138403...",0.50,18,...,"[0.0681849792599678, 0.004651268012821674, -0....",0.86,40,"[0.03743411973118782, -0.0028653573244810104, ...",0.85,56,"[0.02711622416973114, 0.002749305684119463, 0....",0.86,55,semantic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,THEMENBILDER WÖLFE WOLF IM FREIGELÄNDE WILDPAR...,"[0.007191666401922703, 0.00016177825455088168,...",0.86,56,"[0.0020500654354691505, 0.0032149264588952065,...",0.81,50,"[0.00850240420550108, -0.012211491353809834, -...",0.53,57,...,"[-0.0031316394452005625, -0.029537420719861984...",0.78,14,"[0.03129085898399353, -0.06623951345682144, 0....",0.77,14,"[0.022321928292512894, -0.06995398551225662, 0...",0.77,19,keyword
68,DISKUSSION UM WÖLFE - 1. Apr 2019 in Schorfhei...,"[0.007075883448123932, 0.0181588102132082, 0.0...",0.86,56,"[0.004990954417735338, 0.024247867986559868, 0...",0.82,64,"[0.005959740839898586, 0.008287739008665085, -...",0.52,45,...,"[0.003406079253181815, -0.020622065290808678, ...",0.85,37,"[0.022254712879657745, -0.027742797508835793, ...",0.80,34,"[0.006180736236274242, -0.02586253732442856, 0...",0.81,36,keyword
69,THEMENBILDER WÖLFE WOLF IM FREIGELÄNDE WILDPAR...,"[0.013906391337513924, 0.0011311774142086506, ...",0.85,49,"[0.006578399799764156, 0.00934589747339487, -0...",0.79,19,"[0.014614775776863098, -0.006808537058532238, ...",0.52,45,...,"[-0.0010867614764720201, -0.024665145203471184...",0.79,20,"[0.02821160852909088, -0.06320447474718094, 0....",0.80,34,"[0.019578706473112106, -0.06968159973621368, 0...",0.80,33,keyword
70,PK DIE LINKE MIT HARALD WOLF - 13. Aug 2018 in...,"[0.007986604236066341, 0.014006776735186577, -.

In [2]:
import google.auth
from google.auth import exceptions
from google.auth.transport import requests as grequests

CHECK_GROUNDING_PROJECT_ID = "adesso-gcc-docai-demo"

# Get access token
try:
    credentials, project = google.auth.default()
    auth_req = grequests.Request()
    credentials.refresh(auth_req)
    access_token = credentials.token
except exceptions.DefaultCredentialsError:
    print("Please authenticate using gcloud.")

print(f"Authenticated project: {project}")

headers = {
    "Authorization": "Bearer {}".format(access_token),
    "Content-Type": "application/json",
    "X-Goog-User-Project": CHECK_GROUNDING_PROJECT_ID,
}

API_URL = f"https://discoveryengine.googleapis.com/v1alpha/projects/{CHECK_GROUNDING_PROJECT_ID}/locations/global/rankingConfigs/default_ranking_config:rank"

Authenticated project: None


In [29]:
from curses.ascii import EM
import re
import requests

SAMPLE_FILE = EMBEDDING_EVAL_DIR / "Angriff_Wolf.parquet"


def split_title_content(text: str) -> tuple[str, str]:
    match = re.match(r"(.*? - \d{1,2}\. \w{3} \d{4} in .*?)\n(.*)", text, re.DOTALL)
    if match:
        title = match.group(1)
        content = match.group(2)
    else:
        title = text.split("\n")[0]
        content = "\n".join(text.split("\n")[1:])
    return title, content


def create_record(text: str, i: int) -> dict[str, str]:
    title, content = split_title_content(text)
    return {"id": str(i), "title": title, "content": content}


# Loop over all files
for filename in EMBEDDING_EVAL_DIR.glob("*.parquet"):
    if filename.name.endswith("_reranked.parquet"):
        continue
    query = " ".join(filename.stem.split("_"))
    df = pd.read_parquet(filename)

    # result column contains text that needs to be split into title and content
    texts = df["result"].tolist()
    records = [create_record(text, i) for i, text in enumerate(texts)]

    data = {
        "model": "semantic-ranker-512@latest",
        "query": query,
        "records": records,
    }

    # Make the POST request.
    response = requests.post(API_URL, headers=headers, data=json.dumps(data))

    # Handle response
    if response.status_code == 200:
        # Successful request, handle the response data
        print(response.json())
    else:
        # Handle error
        print(f"Error: {response.status_code}, {response.text}")

    response_data = response.json()
    rerank_df = pd.DataFrame(
        {
            "rerank_order": [i + 1 for i in range(len(response_data["records"]))],
            "original_id": [int(record["id"]) for record in response_data["records"]],
            "rerank_distance": [
                float(record["score"]) for record in response_data["records"]
            ],
        }
    )

    rerank_df = rerank_df.sort_values("original_id").set_index("original_id", drop=True)
    print(rerank_df)
    df = df.join(rerank_df)
    df = clean_df(df).rename(columns={"rerank_distance": "rerank_score"})

    df.to_parquet(EMBEDDING_EVAL_DIR / f"{filename.stem}_reranked.parquet")

{'records': [{'id': '60', 'title': 'DEUTSCHE BAHN SETZT TEILHABECHANCENGESETZ ERFOLGREICH UM - 20. Jan 2020 in Berlin, BRD', 'content': 'Gruppenbild Mitarbeiter Deutsche Bahn + ehemalige Langzeitarbeitslose, auch mit Hubertus Heil, Daniel Terzenbach und Martin Seiler (div.)\n---\nWIRTSCHAFT;POLITIK;ARBEITSPLATZ;DEUTSCHE BAHN;ARBEITNEHMER', 'score': 0.12}, {'id': '43', 'title': 'FEUER AN BÖSCHUNG BAHNDAMM GREIFT AUF WOHNSIEDLUNG ÜBER - 7. Aug 2018 in Siegburg, BRD', 'content': 'Mitarbeiter LKA und Polizei gegenüber von Unglücksort Böschungsbrand, der auf andere Seite der Bahngleise und Häuser übergriff\n---\nUNGLÜCK;HITZE;BRAND;BÖSCHUNG;HAUS;DEUTSCHE BAHN;BAHNDAMM;HITZE', 'score': 0.11}, {'id': '57', 'title': 'ZEUGEN IN IMPEACHMENT-ERMITTLUNGEN ERNEUERN KRITIK AN TRUMP-TELEFONAT - 19. Nov 2019 in Washington, USA', 'content': 'Ankunft Tim Morrison, ehemaliger Mitarbeiter Nationaler Sicherheitsrat (div.)\n---\nPOLITIK;AMTSENTHEBUNGSVERFAHREN;ANHÖRUNG;SKANDAL;KONGRESS', 'score': 0.1}, {'id

/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[original_rank_col] = df.index + 1
/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[result_col + "_result"] = helper_df["result"].values
/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

{'records': [{'id': '4', 'title': 'SPD OSTKONVENT - 30. Mai 2021 in Halle, BRD', 'content': 'Olaf Scholz hält Rede auf Bühne\n---\nPOLITIK;SPD;OSTDEUTSCHLAND;WAHLKAMPF', 'score': 0.83}, {'id': '7', 'title': 'LANDTAGSWAHL NIEDERSACHSEN, WAHLKAMPFABSCHLUSS SPD MIT OLAF SCHOLZ - 8. Oct 2022 in Hannover, BRD', 'content': 'Olaf Scholz geht ans Rednerpult, Beginn Rede OT Olaf Scholz, Bundeskanzler (zwischendurch OT Pfiffe, lautstarke Zwischenrufe durch Störer)\n---\nPOLITIK;LTW NIEDERSACHSEN;LTW NIEDERSACHSEN;WAHLKAMPF', 'score': 0.78}, {'id': '5', 'title': 'SPD WAHLKAMPF MIT OLAF SCHOLZ - 19. Sep 2021 in Nürnberg, BRD', 'content': 'Applaus, Publikum,, Olaf Scholz bei Rede auf Bühne\n---\nPOLITIK;BTW 2021;BTW 2021;WAHLKAMPF;SPD', 'score': 0.6}, {'id': '1', 'title': 'BUNDESKANZLER OLAF SCHOLZ HÄLT GRUNDSATZREDE VOR EUROPAPARLAMENT - 9. Mai 2023 in Straßburg, Frankreich', 'content': 'Olaf Scholz hält Rede im EU-Parlament\n---\nPOLITIK;EU;REDE', 'score': 0.58}, {'id': '29', 'title': 'BUNDESFINA

/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[original_rank_col] = df.index + 1
/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[result_col + "_result"] = helper_df["result"].values
/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

{'records': [{'id': '0', 'title': 'TOTALE SONNENFINSTERNIS BEGEISTERT MENSCHEN - 2. Jul 2019 in *, Chile', 'content': 'Blick auf Stadt in Tal während totaler Sonnenfinsternis, Menschen jubeln, verdeckte Sonne mit Silhouetten von Personen, Dämmerung nach Sonnenfinsternis\n---\nUMWELT;SONNENFINSTERNIS', 'score': 0.2}, {'id': '3', 'title': 'TOTALE SONNENFINSTERNIS BEGEISTERT MENSCHEN - 2. Jul 2019 in La Silla, Chile', 'content': 'Menschen beobachten mit Ferngläsern und Sonnenfinsternisbrillen den Himmel, OT Sebastian Pinera, Präsident von Chile\nzu Sonnenfinsternis\n---\nUMWELT;SONNENFINSTERNIS', 'score': 0.13}, {'id': '1', 'title': 'TOTALE SONNENFINSTERNIS BEGEISTERT MENSCHEN - 2. Jul 2019 in Chascomús, Argentinien', 'content': 'Menschen in Winterbekleidung beobachten mit Kameras und Sonnenfinsternisbrillen den Himmel, OT Beobachter, Menschen stehen im Sonnenuntergang auf einem Steg\n---\nUMWELT;SONNENFINSTERNIS', 'score': 0.13}, {'id': '2', 'title': 'TOTALE SONNENFINSTERNIS BEGEISTERT M

...

/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[original_rank_col] = df.index + 1
/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[result_col + "_result"] = helper_df["result"].values
/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

{'records': [{'id': '74', 'title': 'ANKLAGE GEGEN DONALD TRUMP WEGEN STURM AUF DAS KAPITOL ERHOBEN - 1. Aug 2023 in , ', 'content': 'Donald Trump, ehemaliger US-Präsident, Post zu Anklage wegen u.a. Verschwörung und den Sturm auf das Kapitol, auf Truth Social\n---\nPOLITIK;KRIMINALITÄT;ANKLAGE TRUMP;ANKLAGE;KLAGE;WAHLBETRUG;AUFHETZUNG;AUFWIEGELUNG;VERSCHWÖRUNG', 'score': 0.14}, {'id': '76', 'title': 'VERTEILUNG QANON-SCHAMANE IN PROZESS NACH STURM AUF DAS CAPITOL - 17. Nov 2021 in Washington, USA', 'content': "U.S. Capitol rioter 'QAnon Shaman' gets over three years in prison, Gerichtszeichungen\n---\nTERRORISMUS;PROZESS;STURM;KAPITOL;PROZESS;URTEIL", 'score': 0.14}, {'id': '26', 'title': 'PROTESTE CAPITOL - WEITERES MATERIAL VON DER ERSTÜRMUNG - 6. Jan 2021 in Washington, USA', 'content': 'Erstürmung, Trumg Anhänger klettern Mauer am Kapitol hoch\n---\nKONFLIKT;POLITIK;PROTESTE CAPITOL;PROTESTE CAPITOL;ERSTÜRMUNG;KRAWALL;GEWALT', 'score': 0.13}, {'id': '33', 'title': 'PROTESTE CAPITOL

/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[original_rank_col] = df.index + 1
/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[result_col + "_result"] = helper_df["result"].values
/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

{'records': [{'id': '46', 'title': 'SITZUNG BUNDESTAG - 22. Sep 2022 in Berlin, BRD', 'content': 'Christian Lindner und Robert Habeck sitzem beisammen und lachen (Emotion, Freude)\n---\nPOLITIK;BUNDESTAG;SITZUNG', 'score': 0.88}, {'id': '44', 'title': 'BRD/BERLIN:ABREISE BUNDESKANZLER SCHOLZ UND BUNDESMINISTER ZU JAPANREISE - 17. Mar 2023 in Berlin, BRD', 'content': 'Christian Lindenr, Robert Habeck, betreten Flugzeug, lachen\n---\nPOLITIK;UNBEKANNT', 'score': 0.83}, {'id': '41', 'title': 'MINISTER ERHALTEN ÄPFEL + KABINETTSSITZUNG - 5. Oct 2022 in Berlin, BRD', 'content': 'Hubert Heil, Bundesarbeitsminister (lacht, Mimik) im Gespräch mit Robert Habeck\n---\nPOLITIK;KABINETTSSITZUNG', 'score': 0.68}, {'id': '7', 'title': 'ZUSAMENTRITT PARTEIRAT BÜNDNIS 90 DIE GRÜNEN + REAKTION ZU AKTUELLEN POLITISCHEN THEMEN - 12. Nov 2018 in Berlin, BRD', 'content': 'Robert Habeck im Gespräch mit Michael Kellner\n---\nPOLITIK;PARTEISPENDE AFD;REAKTION;RÜCKZUG SEEHOFER;PARTEISPENDE AFD', 'score': 0.48}

/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[original_rank_col] = df.index + 1
/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[result_col + "_result"] = helper_df["result"].values
/Users/philipp.schoneville/Projects/tests/vertex-ai-embedding-text/src/common.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [31]:
with pd.ExcelWriter(DATA_DIR / "results_reranked.xlsx") as writer:
    for data_file in EMBEDDING_EVAL_DIR.glob("*_reranked.parquet"):
        df = pd.read_parquet(data_file)
        df.to_excel(writer, sheet_name=data_file.stem[:30], index=False)